In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.


import libraries

In [3]:
#import libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import spacy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\65977\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\65977\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


load dataset

In [64]:
# Load the dataset
data = pd.read_csv('../IT1244-Project/Twitter Sentiment/dataset.csv', header=None)
data.columns = ['labels', 'text']
print(len(data))
print(data.head())

100000
   labels                                               text
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  is upset that he can't update his Facebook by ...
2       0  @Kenichan I dived many times for the ball. Man...
3       0    my whole body feels itchy and like its on fire 
4       0  @nationwideclass no, it's not behaving at all....


Data preprocessing

We clean the data by removing url mentions hashtags and keeping only the letters then convert to lower case

Then we tokenise the text then we remove stop words and lanel the data

In [65]:
# Clean and preprocess the text data
import spacy

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^A-Za-z]+', ' ', text)  # Keep only letters
    text = text.lower()  # Convert to lowercase
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

# Tokenize the text
data['tokens'] = data['cleaned_text'].apply(word_tokenize)

# Stemming the individual words
stemmer = PorterStemmer()

def stem_words(word_list):
    return [stemmer.stem(word) for word in word_list]

data['stemmed_tokens'] = data['tokens'].apply(stem_words)

# Lemmatizating the individual words 

nlp = spacy.load("en_core_web_sm")

def lemmatize_words(word_list):
    doc = nlp(" ".join(word_list))  # Join the stemmed tokens into a string
    lemmatized_text = [token.lemma_ for token in doc]
    return lemmatized_text

data['lemmatized_token'] = data['stemmed_tokens'].apply(lemmatize_words)

# Remove stop words
stop_words = set(stopwords.words('english'))
#data['filtered_tokens'] = data['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

data.head()

KeyboardInterrupt: 

In [ ]:
stop_words = set(stopwords.words('english'))
data['filtered_tokens'] = data['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

data.head()


,labels,text,cleaned_text,tokens,stemmed_tokens,lemmatized_token,filtered_tokens
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww that s a bummer you shoulda got david ca...,"[awww, that, s, a, bummer, you, shoulda, got, ...","[awww, that, s, a, bummer, you, shoulda, got, ...","[awww, that, s, a, bummer, you, shoulda, get, ...","[awww, bummer, shoulda, got, david, carr, thir..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...,"[is, upset, that, he, can, t, update, his, fac...","[is, upset, that, he, can, t, updat, hi, faceb...","[be, upset, that, he, can, t, updat, hi, faceb...","[upset, update, facebook, texting, might, cry,..."
2,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sa...,"[i, dived, many, times, for, the, ball, manage...","[i, dive, mani, time, for, the, ball, manag, t...","[I, dive, mani, time, for, the, ball, manag, t...","[dived, many, times, ball, managed, save, rest..."
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[my, whole, bodi, feel, itchi, and, like, it, ...","[my, whole, bodi, feel, itchi, and, like, it, ...","[whole, body, feels, itchy, like, fire]"
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behaving at all i m mad why am i ...,"[no, it, s, not, behaving, at, all, i, m, mad,...","[no, it, s, not, behav, at, all, i, m, mad, wh...","[no, it, s, not, behav, at, all, I, m, mad, wh...","[behaving, mad, see]"


In [58]:
# One hot encoding
categories = list(set([item for sublist in data['filtered_tokens'] for item in sublist]))
vectorizer = CountVectorizer(binary=True, vocabulary=categories)
sentences = [' '.join(words) for words in data['filtered_tokens']]

# Fit and transform the sentences
one_hot_encoded = vectorizer.fit_transform(sentences)

# Convert the one-hot encoded data to a DataFrame
encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns = categories)
#encoded_df['labels'] = data['labels']

# use the print function to check if the one hot encoding is correct
#print(encoded_df.iloc[954,:].sum(),sentences[954]) 
print(encoded_df)



      lucu  ta  baltimore  worry  rp  pour  grog  esteem  algo  marriages  \
0        0   0          0      0   0     0     0       0     0          0   
1        0   0          0      0   0     0     0       0     0          0   
2        0   0          0      0   0     0     0       0     0          0   
3        0   0          0      0   0     0     0       0     0          0   
4        0   0          0      0   0     0     0       0     0          0   
...    ...  ..        ...    ...  ..   ...   ...     ...   ...        ...   
9995     0   0          0      0   0     0     0       0     0          0   
9996     0   0          0      0   0     0     0       0     0          0   
9997     0   0          0      0   0     0     0       0     0          0   
9998     0   0          0      0   0     0     0       0     0          0   
9999     0   0          0      0   0     0     0       0     0          0   

      ...  pace  flashden  chars  chastised  tourney  wasted  disappointed 

In [59]:
count = (data['labels'] == 0).sum()
print(count)


5000


In [60]:
# Split the dataset

X_train, X_test, y_train, y_test=train_test_split(encoded_df, data['labels'], stratify=data['labels'], test_size=0.2, random_state=42)



In [61]:
print(y_train)
sum(y_train)

2858     0
96842    1
3905     0
97684    1
1564     0
        ..
99987    1
1717     0
98795    1
3691     0
131      0
Name: labels, Length: 8000, dtype: int64


4000

In [62]:
# Naive Bayes
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix  # Import the confusion_matrix function
from sklearn.naive_bayes import BernoulliNB
#X_train, X_test, Y_train, Y_test

def model_Evaluate(model):
    y_pred = model.predict(X_test)
    print(classification_report(Y_test, y_pred)) 
    return y_pred 
#Precision: ratio of true positives to the total number of predicted positives
#Recall: ratio of true positives to the total number of actual positives
#F1-Score: mean of precision and recall
#Support: number of occurrences of each class in the true labels (y_test).


#confuion matrix, TP,TN, FP, FN

BNB = BernoulliNB()
BNB.fit(X_train, y_train)
y_pred = BNB.predict(X_test)
cf_matrix = confusion_matrix(y_test, y_pred) 
#TP,FN, FP, TN in %%%% 
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
print(classification_report(y_test, y_pred)) 
print(group_percentages)



              precision    recall  f1-score   support

           0       0.73      0.72      0.73      1000
           1       0.73      0.74      0.73      1000

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

['36.05%', '13.95%', '13.00%', '37.00%']


then we do bigram analysos

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an N-gram model (e.g., bigrams)
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))
X_train_ngrams = ngram_vectorizer.fit_transform(X_train)
X_test_ngrams = ngram_vectorizer.transform(X_test)


use a multinomial nb classifier

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Train a classifier (e.g., Multinomial Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train_ngrams, Y_train)

# Make predictions
y_pred = classifier.predict(X_test_ngrams)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
classification_rep = classification_report(Y_test, y_pred, target_names=le.classes_)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')


ValueError: Found input variables with inconsistent numbers of samples: [2791, 800]

In [ ]:
#test